In [1]:
import os
import gc

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 200)

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = '../data'
data_filepath = os.path.join(data_dir, 'harddrive_preprocessed')
CYCLE_ID_FILEPATH = os.path.join(data_dir, 'cycle_id.csv')
CYCLE_ID_FAILURE_FILEPATH = os.path.join(data_dir, 'cycle_id_failure.csv')
os.path.exists(data_filepath)

True

## Load data with PySpark session

In [3]:
spark = (SparkSession.builder
         .config("spark.driver.memory", "12g")
         .appName("SparkSQL").getOrCreate())

your 131072x1 screen size is bogus. expect trouble
24/08/24 13:40:26 WARN Utils: Your hostname, cedric-yu-work resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/24 13:40:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/24 13:40:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
date_col = 'date'
target_label = 'failure'
id_cols = ['serial_number', 'model']
cycle_id_col = 'cycle_id'


In [5]:
df = (spark.read.option("header", "true")
      .option("inferSchema", "true")
      .csv(data_filepath))
df = df.sort(id_cols + [date_col])

df_cycle_id = (spark.read.option("header", "true")
      .option("inferSchema", "true")
      .csv(CYCLE_ID_FILEPATH)).sort(id_cols + [date_col])

df_cycle_id_failure = (spark.read.option("header", "true")
      .option("inferSchema", "true")
      .csv(CYCLE_ID_FAILURE_FILEPATH)).sort(id_cols + [date_col])

In [12]:
# only keep failed cycles for training
df_training = df.join(
    df_cycle_id_failure,
    on=id_cols + [date_col], how='inner'
).sort(id_cols + [date_col, cycle_id_col])

# drop constant columns
col_value_counts = df_training.agg(*(F.countDistinct(F.col(c)).alias(c)
                                     for c in df_training.columns)).cache()
df_col_value_counts = col_value_counts.toPandas()
const_cols = list(
    df_col_value_counts[df_col_value_counts < 2].T.dropna().index)
const_cols = [col for col in const_cols
              if col not in id_cols + [date_col, target_label, cycle_id_col]]
df_training = df_training.drop(*const_cols)


24/08/24 13:42:32 WARN CacheManager: Asked to cache already cached data.


In [13]:
df_training.show()

+-------------+------------------+----------+-------+--------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+-------------+--------+
|serial_number|             model|      date|failure|      capacity_bytes|smart_1_normalized|smart_1_raw|smart_3_normalized|smart_3_raw|smart_4_normalized|smart_4_raw|smart_5_normalized|smart_5_raw|smart_7_normalized|smart_7_raw|smart_9_normalized|smart_9_raw|smart_10_normalized|smart_10_raw|smart_12_normalized|smart_12_raw|smart_188_raw|smart_192_normalized|smart_192_raw|smart_193_normalized|smart_193_raw|smart_194

In [15]:
num_lags = 3
sort_cols = id_cols + [date_col]

# lag features
no_lag_cols = id_cols + [date_col, target_label, cycle_id_col]
window = Window.partitionBy(cycle_id_col).orderBy(date_col)
for col in df_training.columns:
    if col in no_lag_cols:
        continue
    for n in range(num_lags):
        df_training = df_training.withColumn(col + f"_lag_{n+1}",
                          F.lag(col, offset=n+1).over(window))
df_training = df_training.dropna().sort(sort_cols)


In [16]:
df_training.show()

+--------------+--------------------+----------+-------+--------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+-------------+--------+--------------------+--------------------+--------------------+------------------------+------------------------+------------------------+-----------------+-----------------+-----------------+------------------------+------------------------+------------------------+-----------------+-----------------+-----------------+------------------------+------------------------+------------------------+-----------------+---

In [5]:
df = (spark.read.option("header", "true")
      .option("inferSchema", "true")
      .csv(data_filepath))
df = df.sort(id_cols + [date_col])

df_cycle_id = (spark.read.option("header", "true")
      .option("inferSchema", "true")
      .csv(CYCLE_ID_FILEPATH)).sort(id_cols + [date_col])

df_cycle_id_failure = (spark.read.option("header", "true")
      .option("inferSchema", "true")
      .csv(CYCLE_ID_FAILURE_FILEPATH)).sort(id_cols + [date_col])

In [6]:
df_cycle_id.count()

3058798

In [7]:
df_cycle_id_failure.count()

2757

In [7]:
df.show(10)

24/08/24 12:02:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------+-------------+------------------+--------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+
|      date|failure|serial_number|             model|      capacity_bytes|smart_1_normalized|smart_1_raw|smart_3_normalized|smart_3_raw|smart_4_normalized|smart_4_raw|smart_5_normalized|smart_5_raw|smart_7_normalized|smart_7_raw|smart_9_normalized|smart_9_raw|smart_10_normalized|smart_10_raw|smart_12_normalized|smart_12_raw|smart_188_raw|smart_192_normalized|smart_192_raw|smart_193_normalized|smart_193_r

In [8]:
df.count()

3058798

In [8]:
gc.collect()

161

In [10]:
df.groupby([target_label]).count().show()

+-------+-------+
|failure|  count|
+-------+-------+
|      1|    205|
|      0|3058593|
+-------+-------+



In [11]:
df_cycle_id_failure.show()

+-------------+------------------+----------+--------+
|serial_number|             model|      date|cycle_id|
+-------------+------------------+----------+--------+
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-01|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-02|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-03|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-04|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-05|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-06|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-07|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-08|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-09|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-10|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-11|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-12|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-13|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-14|       1|
|    13H2B97AS|TOSHIBA DT01ACA300|2016-01-15|       1|
|    13H2B

## Keep failed cycles only for training

In [12]:
df1 = df.join(
    df_cycle_id_failure,
    on=id_cols + [date_col], how='inner'
).sort(id_cols + [date_col, cycle_id_col])

In [13]:
df1.show()

+-------------+------------------+----------+-------+--------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+--------+
|serial_number|             model|      date|failure|      capacity_bytes|smart_1_normalized|smart_1_raw|smart_3_normalized|smart_3_raw|smart_4_normalized|smart_4_raw|smart_5_normalized|smart_5_raw|smart_7_normalized|smart_7_raw|smart_9_normalized|smart_9_raw|smart_10_normalized|smart_10_raw|smart_12_normalized|smart_12_raw|smart_188_raw|smart_192_normalized|smart_192_raw|smart_193_normalized|sm

In [14]:
df1.groupby([target_label]).count().show()

+-------+-----+
|failure|count|
+-------+-----+
|      1|  205|
|      0| 2552|
+-------+-----+



## Column statistics

In [17]:
df1.agg(F.countDistinct(cycle_id_col)).show()

+------------------------+
|count(DISTINCT cycle_id)|
+------------------------+
|                     205|
+------------------------+



Constant columns

In [22]:
col_value_counts = df1.agg(*(F.countDistinct(F.col(c)).alias(c) for c in df1.columns)).cache()
col_value_counts.show()

24/08/24 12:23:03 WARN CacheManager: Asked to cache already cached data.


+-------------+-----+----+-------+--------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+--------+
|serial_number|model|date|failure|capacity_bytes|smart_1_normalized|smart_1_raw|smart_3_normalized|smart_3_raw|smart_4_normalized|smart_4_raw|smart_5_normalized|smart_5_raw|smart_7_normalized|smart_7_raw|smart_9_normalized|smart_9_raw|smart_10_normalized|smart_10_raw|smart_12_normalized|smart_12_raw|smart_188_raw|smart_192_normalized|smart_192_raw|smart_193_normalized|smart_193_raw|smart_194_normalized|smart_194_raw|sma

In [35]:
df_col_value_counts = col_value_counts.toPandas()

In [46]:
const_cols = list(df_col_value_counts[df_col_value_counts < 2].T.dropna().index)
const_cols = [col for col in const_cols
              if col not in id_cols + [date_col, target_label, cycle_id_col]]
const_cols

['smart_199_normalized']

In [47]:
df1.select(*const_cols).show()

+--------------------+
|smart_199_normalized|
+--------------------+
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
|                 200|
+--------------------+
only showing top 20 rows



In [67]:
num_cols = [c for c, t in df1.dtypes if t.startswith('string')==False]

In [91]:
summary = df1.select(*num_cols).drop(cycle_id_col).summary().cache()
summary.show()

+-------+-------------------+--------------------+------------------+-------------------+------------------+-----------------+--------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+--------------------+--------------------+------------------+-------------------+--------------------+------------------+--------------------+-----------------+--------------------+-----------------+--------------------+------------------+--------------------+------------------+--------------------+-----------------+--------------------+------------------+--------------------+
|summary|            failure|      capacity_bytes|smart_1_normalized|        smart_1_raw|smart_3_normalized|      smart_3_raw|  smart_4_normalized|       smart_4_raw|smart_5_normalized|       smart_5_raw|smart_7_normalized|      smart_7_raw|smart_9_normalized|       smart_9_raw|smart_10_normalized|        smart_10_raw| 

In [76]:
mean_by_target = (df1.groupby(target_label)
                  .agg(*(F.mean(F.col(c)).alias(c)
                         for c in num_cols if c not in [date_col, cycle_id_col, target_label]))
                         ).cache()

In [77]:
mean_by_target.show()

+-------+--------------------+------------------+-------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+----------------+------------------+------------------+-------------------+--------------------+-------------------+------------------+----------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+------------------+--------------------+-----------------+------------------+
|failure|      capacity_bytes|smart_1_normalized|        smart_1_raw|smart_3_normalized|       smart_3_raw|smart_4_normalized|      smart_4_raw|smart_5_normalized|       smart_5_raw|smart_7_normalized|     smart_7_raw|smart_9_normalized|       smart_9_raw|smart_10_normalized|        smart_10_raw|smart_12_normalized|      smart_12_raw|   smart_188

## Drop constant columns from training set

In [93]:
col_value_counts = df1.agg(*(F.countDistinct(F.col(c)).alias(c) for c in df1.columns)).cache()
df_col_value_counts = col_value_counts.toPandas()
const_cols = list(df_col_value_counts[df_col_value_counts < 2].T.dropna().index)
const_cols = [col for col in const_cols
              if col not in id_cols + [date_col, target_label, cycle_id_col]]
df1 = df1.drop(*const_cols)

24/08/24 13:15:21 WARN CacheManager: Asked to cache already cached data.


In [95]:
const_cols

['smart_199_normalized']

## Lag features by cycle_id

In [104]:
num_lags = 3

In [107]:
df1_test = df1.limit(10).cache()
df1_test.show()

+-------------+------------------+----------+-------+--------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+-------------+--------+
|serial_number|             model|      date|failure|      capacity_bytes|smart_1_normalized|smart_1_raw|smart_3_normalized|smart_3_raw|smart_4_normalized|smart_4_raw|smart_5_normalized|smart_5_raw|smart_7_normalized|smart_7_raw|smart_9_normalized|smart_9_raw|smart_10_normalized|smart_10_raw|smart_12_normalized|smart_12_raw|smart_188_raw|smart_192_normalized|smart_192_raw|smart_193_normalized|smart_193_raw|smart_194

24/08/24 13:25:07 WARN CacheManager: Asked to cache already cached data.


In [110]:
no_lag_cols = id_cols + [date_col, target_label, cycle_id_col]
window = Window.partitionBy(cycle_id_col).orderBy(date_col)
for col in df1.columns:
    if col in no_lag_cols:
        continue
    for n in range(num_lags):
        df1 = (df1.withColumn(
            col +
            f"_lag_{n+1}", F.lag(col, offset=n+1).over(window)
        ))

In [111]:
df1.show()

+-------------+------------------+----------+-------+--------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+-------------+--------+--------------------+--------------------+--------------------+------------------------+------------------------+------------------------+-----------------+-----------------+-----------------+------------------------+------------------------+------------------------+-----------------+-----------------+-----------------+------------------------+------------------------+------------------------+-----------------+------

In [94]:
df1.show()

+-------------+------------------+----------+-------+--------------------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+------------------+-----------+-------------------+------------+-------------------+------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+--------------------+-------------+-------------+-------------+-------------+-------------+--------+
|serial_number|             model|      date|failure|      capacity_bytes|smart_1_normalized|smart_1_raw|smart_3_normalized|smart_3_raw|smart_4_normalized|smart_4_raw|smart_5_normalized|smart_5_raw|smart_7_normalized|smart_7_raw|smart_9_normalized|smart_9_raw|smart_10_normalized|smart_10_raw|smart_12_normalized|smart_12_raw|smart_188_raw|smart_192_normalized|smart_192_raw|smart_193_normalized|smart_193_raw|smart_194

In [17]:
spark.stop()